In [ ]:
import numpy as np
import spkit as sp
import pyxdf
from sklearn.cross_decomposition import CCA

In [ ]:
def getReferenceSignals(length, target_freq, samplingRate):
# generate sinusoidal reference templates for CCA for the first and second harmonics
    reference_signals = []
    t = np.arange(0, (length/(samplingRate)), step=1.0/(samplingRate))

    #First harmonics/Fundamental freqeuncy
    reference_signals.append(np.sin(np.pi*2*target_freq*t))
    reference_signals.append(np.cos(np.pi*2*target_freq*t))
    
    #Second harmonics
    reference_signals.append(np.sin(np.pi*4*target_freq*t))
    reference_signals.append(np.cos(np.pi*4*target_freq*t))

    #Third harmonics
    #reference_signals.append(np.sin(np.pi*8*target_freq*t))
    #reference_signals.append(np.cos(np.pi*8*target_freq*t))

    reference_signals = np.array(reference_signals)
    return reference_signals
    
def findCorr(n_components,numpyBuffer,freq):
# Perform Canonical correlation analysis (CCA)
# numpyBuffer - EEG data
# freq - set of sinusoidal reference templates corresponding to the flicker frequency
    cca = CCA(n_components)
    corr=np.zeros(n_components)
    result=np.zeros((freq.shape)[0])
    
    for freqIdx in range(0,(freq.shape)[0]):
        
        cca.fit(numpyBuffer,freq[freqIdx,:,:].T)
        O1_a,O1_b = cca.transform(numpyBuffer, freq[freqIdx,:,:].T)
        indVal=0
        for indVal in range(0,n_components):
            corr[indVal] = np.corrcoef(O1_a[:,indVal],O1_b[:,indVal])[0,1]

        result[freqIdx] = np.max(corr)

    return result


In [ ]:
# read data
data1,_ =pyxdf.load_xdf("C:\\Users\\uwnrelab\\Dalya\\cognixion\\eyeTrackTest\\sub-Erica_ses-Hybrid_2_task-Default_run-001_eeg.xdf") 
data2,_ =pyxdf.load_xdf("C:\\Users\\uwnrelab\\Dalya\\cognixion\\newData_threshTest\\sub-D_ses-L_task-Default_run-001_eeg.xdf")
data3,_ =pyxdf.load_xdf("C:\\Users\\uwnrelab\\Dalya\\cognixion\\newData_threshTest\\sub-D_ses-R_task-Default_run-001_eeg.xdf")

y_dataF=[]
for stream in data1:
    #print(stream)
    if stream.get('info').get('name') == ['cognixion_raw_eeg']:
        y_dataF.append(stream.get('time_series'))
y_dataF = y_dataF[0][:, 0:-1]

y_dataL=[]
for stream in data2:
    if stream.get('info').get('name') == ['cognixion_raw_eeg']:
        #print(stream)
        y_dataL.append(stream.get('time_series'))
y_dataL = y_dataL[0][:, 0:-1]

y_dataR=[]
for stream in data3:
    if stream.get('info').get('name') == ['cognixion_raw_eeg']:
        #print(stream)
        y_dataR.append(stream.get('time_series'))
y_dataR = y_dataR[0][:, 0:-1]


In [ ]:
# run CCA to get correlation with each of the reference frequencies
TotalBuffer = y_dataR

win_length = int(2*250)
step_length = int(0.75*250)

frequencies = [7.5, 12, 13] 
samplingRate = 250
Results=[]

# loop through the segments of 2 seconds
for i in range(win_length, len(TotalBuffer), step_length):

    numpyBuffer0 = TotalBuffer[(i-win_length):i, :]
    numpyBuffer = sp.filter_X(numpyBuffer0, band=[5, 30], btype='bandpass', order = 4, fs=samplingRate, verbose=0) 

    #Generate a vector of sinusoidal reference templates for all SSVEP flicker frequencies
    freq = np.empty([len(frequencies), 4, win_length])

    for i in range(0, 3):
        freq[i, :, :]=getReferenceSignals(numpyBuffer.shape[0], frequencies[i], samplingRate)[:, 0:len(numpyBuffer)]

    #Compute CCA 
    n_components=1
    result = findCorr(n_components,numpyBuffer,freq)

    Results.append(result)

print(Results)